In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler

In [5]:

# Path to thermal frames
frames_dir = 'data/images'  # Update this path

# Load all available frames with .csv extension
frame_files = sorted([f for f in os.listdir(frames_dir) if f.startswith('Frame_') and f.endswith('.csv')])

# Initialize feature storage
feature_list = []

# Define a temperature threshold (e.g., 90th percentile for anomaly detection)
threshold_percentile = 90

# Debugging flag
debug = True

# Process each frame
for file_name in frame_files:
    file_path = os.path.join(frames_dir, file_name)
    
    try:
        frame_data = pd.read_csv(file_path, header=None).values  # Load as NumPy array
        frame_data = frame_data.astype(float)  # Ensure numeric data

        if debug:
            print(f"Processing {file_name}: Shape = {frame_data.shape}, Min = {np.min(frame_data)}, Max = {np.max(frame_data)}")

        # Check if frame is empty or only zeros
        if np.all(frame_data == 0):
            print(f"⚠️ Warning: {file_name} contains only zeros. Skipping.")
            continue

        # Compute features
        min_temp = np.min(frame_data)
        max_temp = np.max(frame_data)
        mean_temp = np.mean(frame_data)
        std_temp = np.std(frame_data)
        median_temp = np.median(frame_data)
        q1 = np.percentile(frame_data, 25)
        q3 = np.percentile(frame_data, 75)
        iqr = q3 - q1
        skewness = skew(frame_data.flatten())
        kurt = kurtosis(frame_data.flatten())

        # Compute pixel count above threshold
        threshold_value = np.percentile(frame_data, threshold_percentile)
        high_temp_pixel_count = np.sum(frame_data > threshold_value)

        # Store extracted features
        feature_list.append([file_name, min_temp, max_temp, mean_temp, std_temp, median_temp, q1, q3, iqr, skewness, kurt, high_temp_pixel_count])

    except Exception as e:
        print(f"🚨 Error processing {file_name}: {e}")
        continue  # Skip problematic files

# Create DataFrame
feature_columns = ["Frame", "Min_Temp", "Max_Temp", "Mean_Temp", "Std_Temp", "Median_Temp",
                   "Q1", "Q3", "IQR", "Skewness", "Kurtosis", "High_Temp_Pixels"]
df_features = pd.DataFrame(feature_list, columns=feature_columns)

# 🚀 FIX: Fill NaN values only for numeric columns (excluding "Frame" column)
numeric_cols = df_features.columns[1:]  # Exclude "Frame"
df_features[numeric_cols] = df_features[numeric_cols].fillna(df_features[numeric_cols].mean())

# Normalize numerical features
scaler = MinMaxScaler()
df_features[numeric_cols] = scaler.fit_transform(df_features[numeric_cols])

# Save as CSV for future use
df_features.to_csv("thermal_frame_features_fixed.csv", index=False)
print("✅ Feature extraction completed. Saved as 'thermal_frame_features_fixed.csv'.")



Processing Frame_1.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_10.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_100.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1000.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1001.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1002.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1003.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1004.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1005.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1006.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1007.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1008.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1009.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_101.csv: Shape = (201, 202), Min = nan, Max = nan
Processing Frame_1010.csv: Shape = (201, 202), Min = na

/Users/berkeayyildizli/ENS_Graduation_Project/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/berkeayyildizli/ENS_Graduation_Project/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
